<h1 style='text-align: center;'>Survivor:</h1>
<h2 style='text-align: center;'>Exploring Contestant Trends Through Data</h2>

This projects aims to look at the long-running CBS show Survivor over the years, and determine what demographics of contestants are more likely to win.

We will also be looking to see if there are any trends where certain demographics have lower chances of winning.


In [ ]:
import pandas as pd
import matplotlib as plt
import sqlite3
# Data pulled in/ saved to csv from 2/16/2025

## Importing Data

In [43]:
# I think it will be more difficult than worth to try and get function to create csv if they don't exist
seasons = pd.read_csv('seasons.csv')
contestants = pd.read_csv('contestants.csv')
stats = pd.read_csv('stats.csv')
idols = pd.read_csv('idols.csv')
advantages = pd.read_csv('advantages.csv')
immunities = pd.read_csv('immunities.csv')

## Cleaning Data

In [ ]:
def clean_and_merge(advantages, idols, immunities, stats):
    """
    Cleans tables for advantages, idols and immunities, then merges them to the stats table.
    """
    # drop columns
    advantages = advantages.drop(columns=['Rank', 'Contestant', 'VV', 'VFB', 'Tie broken?'])
    idols = idols.drop(columns=['Rank', 'Contestant'])
    immunities = immunities.drop(columns=['Rank', 'Contestant'])
    # change column names 
    advantages.columns = advantages.columns.str.strip().str.replace('.1', '')
    idols.columns = idols.columns.str.strip().str.replace('.1', '')
    immunities.columns = immunities.columns.str.strip().str.replace('.1', '')
    # strip and replace values (Season column of S, idols table of special characters (*,†/+,#))
    advantages['Season'] = advantages['Season'].str.replace('S', '')
    advantages['Season'] = advantages['Season'].replace({
        'Game Changers': 34,
        'David vs. Goliath': 37,
        'Winners at War': 40,
        'Cambodia': 31,
        'Island of the Idols': 39,
        'HvHvH': 35,
        'Worlds Apart': 30,
        'Kaoh Rong': 32,
        'Ghost Island': 36,
        'urvivor 42': 42,
        'Edge of Extinction': 38,
        'MvGX': 33   
    })
    advantages['Season'] = advantages['Season'].astype(int) # hopefully the other two go more smoothly

    idols['Season'] = idols['Season'].str.replace('S', '') 
    idols['Contestant'] = idols['Contestant'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['IH'] = idols['IH'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['IP'] = idols['IP'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['VV'] = idols['VV'].str.rstrip('†').str.rstrip('#')
    idols = idols.drop(idols[idols['Season'] == '--'].index)
    idols['IH'] = idols['IH'].astype(int)
    idols['IP'] = idols['IP'].astype(int)
    idols['VV'] = idols['VV'].astype(int)
    idols['Season'] = idols['Season'].astype(int)

    immunities['Season'] = immunities['Season'].str.split(':').str[0]
    immunities['Season'] = immunities['Season'].str.strip('Survivor').str.strip('S')
    immunities['Season'] = immunities['Season'].astype(int)
    # merge advantages/idols/immunities together (before merging them to stats?)
    merged_idols = pd.merge(idols, advantages, on=['Contestant', 'Season'], how='outer')
    merged_all = pd.merge(merged_idols, immunities, on=['Contestant', 'Season'], how='outer')
    stats = pd.merge(stats, merged_all, on=['Contestant', 'Season'], how='left')
    # reorder columns to be more readable
    stats = stats[['Season', 'Contestant', 'SurvSc', 'SurvAv', 'ChW', 'ChA', 'ChW%',
                            'SO', 'VFB', 'VAP','TotV','TCA','TC%','wTCR','JVF', 'TotJ', 
                            'JV%', 'IF', 'IH', 'IP', 'VV', 'ICW', 'ICA', 'AF', 'AP', 'Notes']]
    
    return stats.head() # will be used to clean idols, advantages, and immunities. 
# this includes dropping unnecessary columns, stripping values of unnessary charaters, and renaming columns to be more readable
# will likely also use this function to merge all together with stats table, on contestant column(s)
# (i think merging will get rid of duplicate columns the data is merged on. may need to after the fact)
# Also will rename stats table values. after merge?
clean_and_merge(advantages, idols, immunities, stats)
# will need to create and run a pytest for this to ensure it works correctly?

,Season,Contestant,SurvSc,SurvAv,ChW,ChA,ChW%,SO,VFB,VAP,...,JV%,IF,IH,IP,VV,ICW,ICA,AF,AP,Notes
0,1,Kelly Wiglesworth,1.34,12.26,5.87,16.1,0.36,2,6,0,...,0.43,NaN,NaN,NaN,NaN,4.0,8.0,NaN,NaN,NaN
1,1,Richard Hatch,1.58,7.82,1.87,16.1,0.12,0,9,6,...,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Rudy Boesch,1.09,3.95,1.62,15.1,0.11,3,10,8,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Gretchen Cordy,1.12,3.85,1.23,3.07,0.4,0,3,4,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Susan Hawk,0.95,3.67,0.87,15.1,0.06,0,9,5,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def rename_columns(stats):
    """
    
    """
    
    pass
# List of Column name meanings (To potentially update to later):
# -- Stats table --
# SurvSc: Survival Score
# SurvAv: Survival Average
# ChW: Challenge Wins
# ChA: Challenge Appearances
# ChW%: Challenge Win %
# SO: Sit Outs
# VFB: Votes For Bootee
# VAP: Votes Against (Total)
# TotV: Total Votes Cast
# TCA: Tribal Council Appearances
# TC%: Tribal Council %
# wTCR: Tribal Council Ratio (weighted)
# JVF: Jury Votes For 
# TotJ: Total Number Of Jurors
# JV%: Jury Votes %
# -- Idols --
# IF: Idols founds
# IH: Idols held
# IP: Idols Played
# VV: Votes voided
# -- Advantages --
# AF: Advantage found
# AP: Advantage played
# VV: Votes voided - redundant (idol list includes advantagess this is relevant to)
# VFB: Votes for booted player (If adv. extra or steal a vote. Will likely drop)
# Tie broken? Only applies to 2 vote block advantages, will drop
# -- Immunity Wins --
# ICW: Individual (immunity) challenge wins
# ICA: Challenge appearances - similar to stats ChW/ChA (but exclusive to individual)
stats.columns

Index(['Contestant', 'SurvSc', 'SurvAv', 'ChW', 'ChA', 'ChW%', 'SO', 'VFB',
       'VAP', 'TotV', 'TCA', 'TC%', 'wTCR', 'JVF', 'TotJ', 'JV%', 'Season'],
      dtype='object')

### Exploratory Data Analysis